<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#What-is-Text-Fabric?" data-toc-modified-id="What-is-Text-Fabric?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is Text-Fabric?</a></span><ul class="toc-item"><li><span><a href="#Cuneiform-tablets-in-ATF" data-toc-modified-id="Cuneiform-tablets-in-ATF-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Cuneiform tablets in ATF</a></span></li><li><span><a href="#Text-Fabric" data-toc-modified-id="Text-Fabric-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Text-Fabric</a></span></li></ul></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#More-information" data-toc-modified-id="More-information-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>More information</a></span></li><li><span><a href="#Installing-Text-Fabric" data-toc-modified-id="Installing-Text-Fabric-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Installing Text-Fabric</a></span><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#TF-itself" data-toc-modified-id="TF-itself-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>TF itself</a></span></li><li><span><a href="#Cuneiform-data" data-toc-modified-id="Cuneiform-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Cuneiform data</a></span><ul class="toc-item"><li><span><a href="#Tip" data-toc-modified-id="Tip-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Tip</a></span></li></ul></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#A-tour-around-the-TF-API" data-toc-modified-id="A-tour-around-the-TF-API-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>A tour around the TF API</a></span><ul class="toc-item"><li><span><a href="#Sections:-tablet---column---line" data-toc-modified-id="Sections:-tablet---column---line-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Sections: tablet - column - line</a></span></li><li><span><a href="#Navigation" data-toc-modified-id="Navigation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Navigation</a></span><ul class="toc-item"><li><span><a href="#Source-lines" data-toc-modified-id="Source-lines-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Source lines</a></span></li><li><span><a href="#Source-lines:-refined" data-toc-modified-id="Source-lines:-refined-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Source lines: refined</a></span></li><li><span><a href="#Lines-and-cases" data-toc-modified-id="Lines-and-cases-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Lines and cases</a></span></li><li><span><a href="#Line-numbers" data-toc-modified-id="Line-numbers-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Line numbers</a></span></li></ul></li></ul></li><li><span><a href="#Mass-operations" data-toc-modified-id="Mass-operations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Mass operations</a></span><ul class="toc-item"><li><span><a href="#Walking-all-nodes" data-toc-modified-id="Walking-all-nodes-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Walking all nodes</a></span></li><li><span><a href="#All-signs" data-toc-modified-id="All-signs-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>All signs</a></span></li><li><span><a href="#Particular-signs" data-toc-modified-id="Particular-signs-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Particular signs</a></span></li><li><span><a href="#From-instance-to-context" data-toc-modified-id="From-instance-to-context-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>From instance to context</a></span></li><li><span><a href="#Frequency-list" data-toc-modified-id="Frequency-list-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Frequency list</a></span></li></ul></li><li><span><a href="#Structures" data-toc-modified-id="Structures-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Structures</a></span><ul class="toc-item"><li><span><a href="#Quads" data-toc-modified-id="Quads-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Quads</a></span><ul class="toc-item"><li><span><a href="#Getting-a-big-quad" data-toc-modified-id="Getting-a-big-quad-8.1.1"><span class="toc-item-num">8.1.1&nbsp;&nbsp;</span>Getting a big quad</a></span></li><li><span><a href="#Unraveling-a-quad" data-toc-modified-id="Unraveling-a-quad-8.1.2"><span class="toc-item-num">8.1.2&nbsp;&nbsp;</span>Unraveling a quad</a></span></li><li><span><a href="#Collecting-quads-by-depth" data-toc-modified-id="Collecting-quads-by-depth-8.1.3"><span class="toc-item-num">8.1.3&nbsp;&nbsp;</span>Collecting quads by depth</a></span></li><li><span><a href="#Distribution-of-quads-by-depth" data-toc-modified-id="Distribution-of-quads-by-depth-8.1.4"><span class="toc-item-num">8.1.4&nbsp;&nbsp;</span>Distribution of quads by depth</a></span></li></ul></li><li><span><a href="#Clusters" data-toc-modified-id="Clusters-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Clusters</a></span><ul class="toc-item"><li><span><a href="#Kinds-of-clusters" data-toc-modified-id="Kinds-of-clusters-8.2.1"><span class="toc-item-num">8.2.1&nbsp;&nbsp;</span>Kinds of clusters</a></span></li><li><span><a href="#Lengths-of-clusters" data-toc-modified-id="Lengths-of-clusters-8.2.2"><span class="toc-item-num">8.2.2&nbsp;&nbsp;</span>Lengths of clusters</a></span></li></ul></li></ul></li><li><span><a href="#Signs" data-toc-modified-id="Signs-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Signs</a></span><ul class="toc-item"><li><span><a href="#Writing-results-to-file" data-toc-modified-id="Writing-results-to-file-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Writing results to file</a></span></li><li><span><a href="#Prime" data-toc-modified-id="Prime-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Prime</a></span></li><li><span><a href="#Variant" data-toc-modified-id="Variant-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Variant</a></span></li><li><span><a href="#Modifier" data-toc-modified-id="Modifier-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Modifier</a></span></li><li><span><a href="#Full-grapheme-overview" data-toc-modified-id="Full-grapheme-overview-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Full grapheme overview</a></span></li></ul></li><li><span><a href="#End" data-toc-modified-id="End-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>End</a></span></li><li><span><a href="#Search" data-toc-modified-id="Search-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Search</a></span></li><li><span><a href="#Explore-additional-data" data-toc-modified-id="Explore-additional-data-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Explore additional data</a></span></li><li><span><a href="#Add-your-own-data" data-toc-modified-id="Add-your-own-data-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Add your own data</a></span></li><li><span><a href="#Export-to-Emdros-MQL" data-toc-modified-id="Export-to-Emdros-MQL-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Export to Emdros MQL</a></span></li></ul></div>

<img align="right" src="images/quad.png" width="300"/>
<img align="right" src="images/tf-small.png"/>


# Tutorial

This notebook gets you started with using
[Text-Fabric](https://github.com/Dans-labs/text-fabric) for coding in cuneiform tablet transcriptions.

## What is Text-Fabric?

### Cuneiform tablets in ATF

Cuneiform tablets have been transcribed in ATF files, in which the marks on a tablet are represented
by ascii characters. The marks on a tablet have structure (they can be composed, they build cases and lines) and properties (they can be uncertain).

When you search for tablet data in an ATF file, you can do so conveniently by using regular expressions.

However, the ATF transcriptions have become packed with information. Not every transcriber uses ATF in the same way, and there are a few coding errors in the sources.

That means that the most obvious search expressions will leave out cases. Either you live with that, or you refine your search expressions.

An other issue is, that when you look for something, your search expressions must reflect the shape of not
only your target, but also everything else. There is virtually no separation of concerns.

### Text-Fabric

Text-Fabric is a model for textual data with annotations that is optimized for efficient data analysis. Not only that, it also facilitates the creation of new, derived data, which can be added to the original data.
Data combination is a feature of Text-Fabric.

Text-Fabric is being used for the
[Hebrew Bible](https://github.com/ETCBC/bhsa)
and a large body of linguisitic annotations on top of it. The researchers of the 
[ETCBC](http://etcbc.nl) thought that a plain database is not a satisfactory text model,
and that XML is too limited too express multiple hierarchies in a text smoothly.

That's why they adopted a model by
[Doedens](http://books.google.nl/books?id=9ggOBRz1dO4C)
that reflects more of the essential properties of text (sequence, embedding). This model is the basis of MQL, a working text-database system.
Text-Fabric is based on the same
[model](https://github.com/Dans-labs/text-fabric/wiki/Data-model),
and once the data is in Text-Fabric, it can be exported to MQL.

See more on the effort of modeling the Hebrew Bible in Dirk's article
[The Hebrew Bible as Data: Laboratory - Sharing - Experiences](https://doi.org/10.5334/bbi.18)

With data in Text-Fabric, it becomes possible to build rich online interfaces on the data of ancient texts.
For the Hebrew Bible, we have built
[SHEBANQ](https://shebanq.ancient-data.org).

Working with TF is a bit like buying from IKEA. You get your product in bits and pieces, and you assemble it yourself. TF decomposes any dataset into its components, nicely stacked per component, with every component uniquely labeled. You go to the store, make your selection, enter the warehouse, collect your parts, and, at home, assemble your product.

In order to enjoy an IKEA product, you do not need to be a craftsman, but you do need to be able to handle a screw driver.

In the TF world, it is the same. You do not have to be a professional programmer, but you do need to be able to program little things. A first course in Python is enough.

Another parallel: in IKEA you take a package with components home, and there you assemble it. 
In TF it is likewise: you download the TF data, and then you write a little program. Inside that program you can call up the Text-Fabric tool, which act as the IKEA user manual. But your program takes control, not Text-Fabric.

The best environment to enjoy Text-Fabric is in Python programs that you develop in a 
[Jupyter Notebook](http://jupyter.readthedocs.io/en/latest/).
This tutorial is such a notebook. If you are reading it online, you see text bits and code bits,
but you cannot execute the code bits.

If you download this tutorial, and you have installed Python, Jupyter, and Text-Fabric,
you can *execute* the code bits.

## Overview

* we tell you how to get Text-Fabric on your system
* we tell you how to get a set of cuneiform tablet transcriptions on your system
* we show how to explore the data:
  * finding the relevant nodes
  * moving from one place to an other
  * collecting the relevant information
  * perform analysis
  * visualize your results

## More information
Chances are that a bit of reading about the underlying
[data model](https://github.com/Dans-labs/text-fabric/wiki/Data-model)
helps you to follow the exercises below, and vice versa.

We have checked the conversion from the transcriptions to Text-Fabric extensively.
Cruelly, you might say. You can follow the checks
in a separate notebook [checks](checks.ipynb).

## Installing Text-Fabric

### Prerequisites

You need to have Python on your system. Most systems have it out of the box,
but alas, that is python2 and we need at least python 3.6.

Install it from [python.org]() or from [Anaconda]().
If you got it from python.org, you also have to install [Jupyter]().

### TF itself

```
pip install text-fabric
```

if you have installed Python with the help of Anaconda, or

```
sudo -H pip3 install text-fabric
```
if you have installed Python from [python.org](https://www.python.org).

###### Execute: If all this is done, the following cells can be executed.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os, collections
from tf.fabric import Fabric
from utils import Compare

### Cuneiform data

We have prepared a corpus of 6000 tablets, from the Uruk-III/IV period in Text-Fabric.
We have downloaded the transcriptions from CDLI, and converted them to Text-Fabric.

You can get the results from GitHub as follows.

We suggest you make an appropriate directory in your home directory:

```
github/Dans-labs
```

then go to that directory in a terminal, and then say

```
git clone https://github.com/Dans-labs/Nino-cunei
```

After that your directory structure shold look like this:

    your home direcectory\
    |                     - github\
    |                       |      - Dans-labs\
    |                       |        |         - Nino-cunei
    
#### Tip
If you start computing with this tutorial, first copy its parent directory to somewhere else,
outside your `Nino-cunei` directory.
If you pull changes from the `Nino-cunei` repository later, your work will not be overwritten.
Where you put your tutorial directory is up till you.
It will work from any directory.

###### Execute: it this has been done, you can execute the next cells

In [ ]:
REPO = '~/github/Dans-labs/Nino-cunei'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

## Load Data

The transcriptions of the tablets in their TF form is organized in a model of nodes, edges and features.

The things such as tablets, faces, columns, cases, and, at the most basic level, signs, are numbered.
The signs correspond to number 1 ... ca. 120,000, in the same order as they occur in the corpus.
All other things are built from signs. They have number from ca 120,000 to 450,000.

In TF, we call these numbers *nodes*. Like a barcode, this number gives access to a whole bunch of
information about the corresponding object.

For example, lines have a property (in TF we call it a *feature*) called `fullNumber`. 
It contains the hierarchical number found at the start of the line in the transcription.

If the node for a line is `n`, we can find its hierarchical number by saying

```
F.fullNumber.v(n)
```

In words, it reads as:

* `F`: I want to look up a `F`eature
* `fullNumber`: the name of the feature
* `.v`: I want the value of that feature
* `(n)`: for the given node `n`

Seen in this way, the data is like a gigantic spreadsheet of 450,000 rows (the nodes),
with 30 columns (the features).

There is a bit more to it, since the nodes can be grouped together in ways we will see later on.

###### Execute: the next cell loads most features

In [ ]:
api = TF.load('''
    grapheme prime repeat
    variant variantOuter
    modifier modifierInner modifierFirst
    damage uncertain remarkable written
    kind
    period name type identifier catalogId
    number fullNumber origNumber badNumbering
    crossref text
    srcLn srcLnNum
    op sub comments''')
api.makeAvailableIn(globals())
COMP = Compare(api, SOURCE_DIR, TEMP_DIR)

## A tour around the TF API

We have the data of over 6000 tablets at our fingertips now. But how can we see it?
In a meaningful way?

The following tour shows you the most important parts of the Text-Fabric API.
It is a set of functions, defined in the TF package, that you can use in your own program.

This notebook itself is our current program, and we have just loaded the API.

See in the next cells how we navigate to the tablets we want and pick our data from them.

The full API-reference documentation is available on the
[Text-Fabric wiki](https://github.com/Dans-labs/text-fabric/wiki/Api).

### Sections: tablet - column - line

Let's start with a single, specific tablet P448702.

Here is its transcription, copied from the CDLI source.

```
&P448702 = www archaeo-auction 004 
#atf: lang qpc 
@tablet 
@obverse 
@column 1 
$ beginning broken 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
@column 2 
$ beginning broken 
$ blank space 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 
@column 3 
$ beginning broken 
$ blank space 
@reverse 
$ (not imaged) 
```

There is a node for this tablet. How do we find it?

The easiest way is to ask TF to find the node on the basis of a section specification.

When we converted the transcriptions to TF, we made it so that tablets are the first
section level, columns the second, and lines the third. 

In order to find the node corresponding to a section, we have to give a tuple
`(tabletName, columnNumber, lineNumber)` to the appropriate function in TF.

Here you may leave out `lineNumber` (to specify a column), or both `columnNumber` and `lineNumber` 
(to specify a tablet).
The latter is our case here.

In [ ]:
exampleTablet = T.nodeFromSection(('P448702',))
exampleTablet

Ok, this is the node, the barcode, as it were. It gives you access to all data we have stored of this
tablet.

One of the things we have stored for tablets, columns, and lines, is the original transcription line.
Let's retrieve it.

In [ ]:
F.srcLn.v(exampleTablet)

This is the source line of the tablet header.
How do we get the other source lines?

### Navigation

We'll need another concept first.
Things on a tablet have spatial relationships. 
While the transcriptions abstract from the physical relationships, there are still a few
important ones left: sequence and embedding.

At the most basic level we have signs. 
For all other levels, TF knows exactly which signs are part of the objects of those levels.

And based on that knowledge, we can travel from objects to the objects embedded in it, and back,
and to the right and left siblings.

All these functions start with `L.`, and they take a starting node as argument.

* `L.d()` goes "down": from enbedder to embeddee;
* `L.u()` goes "up": from embeddee to embedder;
* `L.p()` goes "previous": to the first left sibling;
* `L.n()` goes "next": to the first right sibling.

We can now find all the nodes contained in the example tablet.
When we retrieve them, we also fetch their *type*: is it a face, a column, a sign?
The type of a node is stored in the feature `otype`. 
This feature is so central to the TF model that every TF dataset has it.

In [ ]:
for child in L.d(exampleTablet):
    print(f'{child:>6} of type {F.otype.v(child)}')

A few things to note:

* the numbers are not really important to us, only to the program.
  soon, we will stop showing them altogether;
* the order in which the nodes are delivered, is a natural one:
  1. first things first, and 
  2. bigger things before the smaller things they contain.

#### Source lines

So in order to find all the source lines of this tablet, we can just take all these nodes,
see whether they have a value for feature *srcLn*, retrieve it, an print it.

Let's do it, and write this as a function, i.e. a recipe that we can apply to every tablet.

In [ ]:
def getSource(tablet):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

This function gets all source lines in a list.
Let's apply it to our sample tablet, and print out the list:

In [ ]:
sourceLines = getSource(exampleTablet)
print('\n'.join(sourceLines))

This is nearly the original. The only difference we spot is that we have left out
the `@tablet` line. This has been done because the `&P` line is the true header of the
tablet. In fact, the `@tablet` line is not consistently present in the source.

As you seen, the TF data is a bit more streamlined than the source transcriptions.

Let's just enjoy the new power, and get the source lines of another tablet: P005381

In [ ]:
sourceLines = getSource(
    T.nodeFromSection(
        ('P005381',)
    )
)
print('\n'.join(sourceLines))

By the way, in column 2, line 1 we find a composition of three levels: 
`|SZU2.((HI+1(N57))+(HI+1(N57)))|`

We call compositions of signs by means of operators such as `.` and `x` and `+` *quads*.
They are also a node type, the first node type above the "atomic" sign.

We'll see a lot of *quads* later on.

#### Source lines: refined

If you just want the lines, and nothing else in between, you can instruct the `L.d` function to
let through only nodes of a certain type.

Let's improve the `getSource` function to give it an additional, optional argument that
tells us the kind of object we want to see.

In [ ]:
def getSource(tablet, nodeType=None):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

If we call it without node type, we get the same results.

Let's call it with `nodeType='face'`:

In [ ]:
sourceLines = getSource(exampleTablet, nodeType='face')
print('\n'.join(sourceLines))

We see only the face specifiers.

Now let's grab the lines: 

In [ ]:
sourceLines = getSource(exampleTablet, nodeType='case')
print('\n'.join(sourceLines))

#### Lines and cases

Note that we actually supplied `nodeType='case'`.
This is because we have modelled the data in such a way, that a line corresponds to all
transcription lines in a column that start with the same number.
Lines are divided in cases and sub-cases according to the hierarchical line numbers.
At the lowest level of sub-case we encounter the material that belong to single
transcription lines.

Also when lines only have a single case, we will have a case embedded in the line.

So *case* is the node type for the geometric divisions of lines,
coded by the hierarchical line number.

#### Line numbers

The TF data also retains the original line numbers, i.e. the line number in the file
that contains the transcriptions.
We do not mean the hierarchical line numbers of the cases!
Let us once again improve the `getSource` function by 
giving it an optional argument for fetching line numbers too:

In [ ]:
def getSource(tablet, nodeType=None, lineNumbers=False):
    sourceLines = []
    if lineNumbers:
        sourceLines.append(f'{F.srcLnNum.v(tablet):>5}: {F.srcLn.v(tablet)}')
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        lineNumber = ''
        if sourceLine:
            if lineNumbers:
                lineNumber = f'{F.srcLnNum.v(child):>5}: '
            sourceLines.append(f'{lineNumber}{sourceLine}')
    return sourceLines

Now we can print the lines with source line numbers, so that we can look them up in the
source easily:

In [ ]:
sourceLines = getSource(exampleTablet, nodeType='case', lineNumbers=True)
print('\n'.join(sourceLines))

You'll appreciate the way you can build up your own power tools as you go.
The functions of TF provide a foundation, so that start when all the gory details
have been taken care of.

Speaking of which: see that strange modifier on `SUKUD`, the `@inversum`?
It is not in the [Oracc primer on ATF](http://oracc.museum.upenn.edu/doc/help/editinginatf/primer/inlinetutorial/index.html)

Probably it is a typo, and the `inversum` should be replaced by a single letter.
While Dirk was programming, he decided to "correct" this into `@v`, but what really is needed here is
to correct the source. Or we could correct the correction.
In either case, we expect to run the conversion again, with all issues like these resolved.

To see what those issues are, see the complete list of
[diagnostics](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/diagnostics.tsv).

## Mass operations

We have a mass of data, and we want to do meaningful things with that mass.
Here are a few ways to address all nodes, and significant subsets of all nodes.

### Walking all nodes

The generator `N()` provides you with all nodes, one by one, in a sequence.
They come in the natural order, as we have seen above:

* first things first
* bigger things before the smaller things they contain.

Let's print the first 20 nodes, together with their types.

In [ ]:
LIMIT = 20

times = 0
for n in N():
    print(f'{n:>6} : {F.otype.v(n)}')
    times += 1
    if times >= 20:
        break

### All signs

Suppose we are only interested in the signs, not how they are organized in cases, lines,
columns, faces and tablets.

Suppose we just want to count them.

We could use `N()` as follows:

In [ ]:
times = 0
for n in N():
    if F.otype.v(n) != 'sign':
        continue
    times += 1
print(times)

In fact, TF makes it a bit easier to get to all sign nodes, or nodes from whatever type.

We have already used the `F.feature.v(n)` idiom.
But you can do more with features, beyond fetching particular values.

You can also retrieve the sequence of nodes that support a particular value.

We want to retrieve all nodes that support the value `sign` in their `otype` feature.

Here we go, and note that we use `.s(value)` instead of `.v(node)`.

In [ ]:
signs = F.otype.s('sign')
len(signs)

Signs have a feature `grapheme`, filled with the name of the sign, without variants (allographs), modifiers, and flags. Let's see the first 20 graphemes.

In [ ]:
for sign in signs[0:20]:
    print(f'{sign:>2} : {F.grapheme.v(sign)}')

A few things to note:

* there are empty signs
* there is the sign `…`

During conversion, we have put empty signs in metadata lines and rulings.
That is the mechanism by which TF knows where in the corpus they are positioned.
They do not belong to the material that corresponds to tablet content, except maybe in the
case of rulings.

The sign `…` originates from `...` in transcriptions, which is mostly surrounded by
`[ ]`. The brackets signify uncertainty, and can be put around other signs as well,
even sequences of signs. These brackets denote *clusters*, a node type that we will encounter
later.

So, the expression `[...]` in the source translates to a *cluster* in TF, with just one
sign in it, which has *grapheme* value `…`. 
Of course, this is not a real grapheme, and it will be easy to weed out such graphemes if
you do not need to take them into account. 

### Particular signs

Suppose we want to get all occurrences of the grapheme `SANGA`.

We can use the same trick as for getting the `signs`: we ask for the nodes that support
the value `SANGA` in their `grapheme` feature.

In [ ]:
sangas = F.grapheme.s('SANGA')
len(sangas)

As an example of what you can do with such a set, let's pick only those SANGAs that
do not have a variant. We want `SANGA`, but not `SANGA~a`.

In [ ]:
pureSangas = [s for s in sangas if F.variant.v(s) is None]
len(pureSangas)

Ah, all sangas are with variant (allograph) in this corpus. Are they all `~a`?

In [ ]:
aSangas = [s for s in sangas if F.variant.v(s) == 'a']
len(aSangas)

Not all of them, let's see what other variants occur:

In [ ]:
sangaVariants = set()
otherSangas = []

for s in sangas:
    if s not in aSangas:
        otherSangas.append(s)
        variant = F.variant.v(s)
        sangaVariants.add(variant)
        print(f'{s:>6} SANGA~{variant}')
print(f'All variants: {", ".join(sorted(sangaVariants))}')

### From instance to context

We want to know where those other SANGA variants are.

Earlier we saw how we could get nodes if we knew the section.
Here we need the opposite. We know a few nodes, and want to get the section.
TF has a function for that:

In [ ]:
for s in otherSangas:
    print(T.sectionFromNode(s))

If we want to see the source code line of these sangas, we can do it by what we have learned so far:

In [ ]:
for s in otherSangas:
    for case in L.u(s, otype='case'):
        srcLine = F.srcLn.v(case)
        if srcLine is not None:
            print(f'{F.srcLnNum.v(case):>5}: {srcLine}')

### Frequency list

TF has a standard function that generates a frequency list for any feature of your choice.

Let's do the modifiers (`@`*mod*):

In [ ]:
F.modifier.freqList()

The result is a tuple of tuples (*modifier*, *amount of occurrences*), ordered in decreasing frequency.
We can print the list a bit more neatly as follows:

In [ ]:
for (value, amount) in F.modifier.freqList():
    print(f'{amount:>4} x {value}')

Note that you can call this for any feature.
So let us do it for graphemes. Note that we get a frequency list for the bare graphemes, ignoring
variants and modifiers. So maybe this is not that useful.

Before we print out the complete list, let's make sure the list is not forbiddingly long.
Or rather, let us print only the top twenty.

In [ ]:
for (value, amount) in F.grapheme.freqList()[0:20]:
    print(f'{amount:>4} x {value}')

## Structures

Some node types represent things with a nested structure. We have seen *quads* above,
an other example are *clusters* of quads. Clusters are called *brackets* in the
[Oracc primer](http://oracc.museum.upenn.edu/doc/help/editinginatf/primer/inlinetutorial/index.html).

In TF, we have two mechanisms to specify that something is a part of an other thing.
One we have already encountered: `L.d(`*node*`)`, which gives you the nodes of all things whose signs
are contained in the signs belonging to *node*.

But for some purposes this is to coarse.
Consider the complex quad we encountered before:

```
|SZU2.((HI+1(N57))+(HI+1(N57)))|
```

This quad has sub-quads `SZU2`, `(HI+1(N57))+(HI+1(N57))`, and the last one
has sub-quads `HI+1(N57)` and `HI+1(N57)`, which have in turn sub-quads `HI` and `1(N57)`.

If we only look at embedding, it is a bit awkward to produce the sub-quads per level, because
if we ask for all sub-quads embedded in the outer quad, we'll get all sub-quads mentioned, regardless
of level. 

So we need an extra device, and TF offers a very generic one for this: *edge features*.

Whereas the *features* we have seen so far, such as `otype`, `grapheme`, and `variant`
annotate nodes, there are also features that annotate pairs of nodes.
A pair of nodes is called an *edge*, and indicates that there is a relationship between nodes.
Edge features give these relationships a name, and can annotate the relationships as well.

We could use edge features to relate lines to other, similar lines in the corpus, and annotate it
with the degree of similarity.

In the case of quads and clusters, we have generated edges in TF between clusters and their components,
and between quads and their components.

### Quads

The components of quads are either sub-quads or signs.
Sub-quads are also quads in TF, and they are always a composition.
Whenever a member of a sub-quad is no longer a composition, it is a *sign*.

Let's try to unravel the structure of this example quad by TF means.

#### Getting a big quad
First we need to get the node of this quad. Above we have seen the source code of the tablet in which
it occurs, from that we can pick the node of the case it is in:

In [ ]:
line = T.nodeFromSection(('P005381', 'obverse:2', '1'))
line

Note, that in order to specify a column, you have to specify it as *face type*`:`*column number*

In [ ]:
case = L.d(line, otype='case')[0]
F.srcLn.v(case)

In order to identify our super-quad, we list all quad nodes that are part of this case.
For every quad we list the node numbers of the signs contained in it.

In order to know what signs are contained in any given node, we use the feature `oslots`.
Like the feature `otype`, this is a standard feature that is always available in a TF dataset.

Unlike `otype`, `oslots` is an edge feature: there is an edge between every node and every slot contained in it.

Whereas you use `F` to do stuff with node features, you use `E` to do business with edge features.

And whereas you use `F.feature.v(node)` to get the feature value of a node, you use 
`E.oslots.s(node)` to get the nodes for which there is an `oslots` edge from `node` to it.

In [ ]:
for node in L.d(case, otype='quad'):
    print(f'{node:>6} {E.oslots.s(node)}')

On the basis of this see what the biggest quad is.
We could have been a bit more friendly to our selves by showing the actual graphemes in the quads.

In [ ]:
for node in L.d(case, otype='quad'):
    print(f'{node:>6} {" ".join(F.grapheme.v(s) for s in E.oslots.s(node))}')

So let us get the node of the biggest quad.

In [ ]:
bigQuad = sorted(
    (
        quad 
        for quad in L.d(case, otype='quad')
    ),
    key = lambda q: -len(E.oslots.s(q))
)[0]
bigQuad

Lo and behold, it is precisely the big quad.

#### Unraveling a quad

Now we are going to retrieve its components by following edges.
We have made `sub` edges from quads to their components.

In order to follow the `sub` edges from a node, you use 

`E.sub.f(node)`.

You can also get the `sub` edges *to* a node:

`E.sub.t(node)`.

In [ ]:
E.sub.f(bigQuad)

Let us unravel the whole structure:

In [ ]:
def unravelQuad(quad):
    if F.otype.v(quad) == 'sign':
        return F.grapheme.v(quad)
    subQuads = E.sub.f(quad)
    unraveledSubQuads = [unravelQuad(subQuad) for subQuad in subQuads]
    return f'<{", ".join(unraveledSubQuads)}>'

unravelQuad(bigQuad)

Later we see how we can get the exact representation of this quad back, including the operators `.` and `+`.

#### Collecting quads by depth

We want to sort all complex quads by the depth of their structure.
In order to do that, we first write a function that computes the depth of a quad.

The idea of that function is:
* if a quad does not have sub-quads (only signs as members), its depth is 1;
* the depth of a quad is 1 more than the maximum of the depths of its sub-quads.

In [ ]:
def depthQuad(quad):
    subDepths = [
        depthQuad(subQuad) 
        for subQuad in E.sub.f(quad) 
        if F.otype.v(subQuad) == 'quad'
    ]
    if len(subDepths) == 0:
        return 1
    else:
        return max(subDepths) + 1

Here is the depth of the big quad:

In [ ]:
depthQuad(bigQuad)

#### Distribution of quads by depth

We want to know how many quads there are, how deeply composed they get, and how often deep compositions
occur.

This is the recipe:
we visit all quads, examine their depths, and tally the appropriate depth.

In [ ]:
depthDistribution = collections.Counter()

for quad in F.otype.s('quad'):
    depth = depthQuad(quad)
    depthDistribution[depth] += 1
    
for (depth, amount) in sorted(
    depthDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x depth {depth}')

It turns out we had spotted the deepest quad already.
Let's finish off by looking which tablets show how many quads with depth >= 2.

The code is much like above, but now we do not tally the depth of a quad, but the tablet
it occurs in.

In [ ]:
tabletQuad = collections.Counter()

for quad in F.otype.s('quad'):
    depth = depthQuad(quad)
    if depth >= 2:
        tablet = L.u(quad, otype='tablet')[0]
        tabletQuad[tablet] += 1

Before we look at the output, let us sort the tablets found on decreasing amount of complex quads:

In [ ]:
tabletQuadList = sorted(
    tabletQuad,
    key=lambda t: (-tabletQuad[t], t),
)

We are not interested in tablets with just one complex quad.

We collect the tablets by their node, but when we output the results, we
want to show the tablet's number, which is stored in the feature `catalogId`.

In [ ]:
singles = 0

for tablet in tabletQuadList:
    amount = tabletQuad[tablet]
    if amount == 1:
        singles += 1
    else:
        print(f'{amount:>5} complex quads in tablet {F.catalogId.v(tablet)}')
print(f'{1:>5} complex quad  in {singles} other tablets')

And, for the record, we'll have a look at the source code of one of the tablets at the top of the list.
We have already defined a function `getSource(tablet)` above, which comes in handy here. 

In [ ]:
sourceLines = getSource(tabletQuadList[0])
print('\n'.join(sourceLines))

### Clusters

Clusters are groups of consecutive quads between brackets.

Clusters can be nested.
As with quads, we find the members of a cluster by following `sub` edges.

#### Kinds of clusters

In our corpus we encounter several types of brackets:

* `( )a` for proper names
* `[ ]` for uncertainty
* `< >` for supplied material.

The first thing is to get on overview of the distribution of these kinds.

In [80]:
clusterKindDistribution = collections.Counter()

for cluster in F.otype.s('cluster'):
    kind = F.kind.v(cluster)
    clusterKindDistribution[kind] += 1

for (kind, amount) in sorted(
    clusterKindDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x a {kind:>8}-cluster')

32321 x a uncertain-cluster
  637 x a properName-cluster
    1 x a supplied-cluster


The conversion to TF has transformed `[...]` to a cluster of one sign with grapheme `…`.
These are trivial clusters and we want to exclude them from further analysis, so we redo the counting.

First we make a sequence of all non-trivial clusters:

In [81]:
realClusters = [
    c 
    for c in F.otype.s('cluster')
    if (
        F.kind.v(c) != 'uncertain' or
        len(E.oslots.s(c)) > 1 or 
        F.grapheme.v(E.oslots.s(c)[0]) != '…'
    )
]
len(realClusters)

3385

Now we redo the same analysis, but we start with the filtered cluster sequence.

In [108]:
clusterKindDistribution = collections.Counter()

for cluster in realClusters:
    kind = F.kind.v(cluster)
    clusterKindDistribution[kind] += 1

for (kind, amount) in sorted(
    clusterKindDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x a {kind:>8}-cluster')

 2747 x a uncertain-cluster
  637 x a properName-cluster
    1 x a supplied-cluster


#### Lengths of clusters

How long are clusters in general?
There are two possible ways to measure the length of a cluster:

* the amount of signs it occupies;
* the amount of top-level members it has (quads or signs)

By now, the pattern to answer questions like this is becoming familiar.

We express the logic in a function, that takes the way of measuring
as a parameter.
In that way, we can easily provide a cluster-length distribution based
on measurements in signs and in quads.

In [121]:
def computeDistribution(nodes, measure):
    distribution = collections.Counter()

    for node in nodes:
        m = measure(node)
        distribution[m] += 1

    for (m, amount) in sorted(
        distribution.items(),
        key=lambda x: (-x[1], x[0]),
    ):
        print(f'{amount:>5} x a measure of {m:>8}')

In [122]:
def lengthInSigns(node):
    return len(L.d(node, otype='sign'))

def lengthInMembers(node):
    return len(E.sub.f(node))

Now we can show the length distributions of clusters by just calling `computeDistribution()`:

In [125]:
computeDistribution(realClusters, lengthInSigns)

 2691 x a measure of        1
  434 x a measure of        2
  205 x a measure of        3
   41 x a measure of        4
    9 x a measure of        5
    3 x a measure of        6
    2 x a measure of        7


In [126]:
computeDistribution(realClusters, lengthInMembers)

 2679 x a measure of        1
  452 x a measure of        2
  194 x a measure of        3
   44 x a measure of        4
   11 x a measure of        5
    4 x a measure of        6
    1 x a measure of        7


If you look closely at the code for these functions, there is nothing in it that 
is specific for clusters.

The measures are in terms of the totally generic `oslots` function, and the fairly generic
`sub` edges, which are also defined for quads.

So, in one go, we can obtain a length distribution of quads.

Note that quads can also be sub-quads.

In [128]:
computeDistribution(F.otype.s('quad'), lengthInSigns)

 3717 x a measure of        2
  181 x a measure of        3
    7 x a measure of        4
    1 x a measure of        5


In [127]:
computeDistribution(F.otype.s('quad'), lengthInMembers)

 3887 x a measure of        2
   19 x a measure of        3


Now we can get the transcription of this tablet.

In [ ]:
transObject(tabletNode)

## Signs

The main characteristic of a sign is its `grapheme`.
Everything we do with signs, is complicated by the fact that signs can be *augmented* with
primes, variants, modifiers and flags.

Below we show how you can get frequency lists of signs, with or without taking
augments into account.

### Writing results to file

We also want to write the results to files in your `_temp` directory, within this repo.
In order to not clutter this notebook, we use a function `writeFreqs()`, defined in 
[utils](utils.py) in the same directory.

We call this function as `COMP.writeFreqs(fileName, dataItems, dataName)`.

`COMP` is an object, created at the start of this notebook, by importing it from the
file `utils.py` in this same directory.
Functions called from `COMP`, know the TF-API, and know where the temp-directory is.

`writeFreqs` writes distribution data of data items called `dataName`
to a file `fileName.txt`.
In fact, it writes two files: 
* `fileName-alpha.txt`, ordered by data items
* `fileName-freq.txt`, ordered by frequency.

In [129]:
COMP.writeFreqs('grapheme-plain', F.grapheme.freqList(), 'bare grapheme')

There are 667 bare graphemes


Now have a look at your TEMP_DIR and you see two generated files:

* `graphemes-plain-alpha.txt` (sorted by grapheme)
* `graphemes-plain-freq.txt` (sorted by frequency)

But we can do better, we also want the prime, variants, and modifiers taken into account.

Let us first see what they can be.

### Prime

The prime is a feature with two values: 1 or 0. 1 means: there is a prime.
Below you see how often that occurs.
Note that we count all primes here: on signs, case numbers and column numbers.

For more info and a check on the occurrences of primes, see [checks](checks.ipynb).

In [130]:
for (value, frequency) in F.prime.freqList():
    print(f'{frequency:>5} x {value}')

 5184 x 1


### Variant

The variant or *allograph* is what occurs after the grapheme and after the `~` symbol, which should be digits and/or
lowercase letters except the `x`.

Here is the frequency list of variant values.

In [131]:
for (value, frequency) in F.variant.freqList():
    print(f'{frequency:>5} x {value}')

23804 x a
 4172 x b
 1532 x c
 1356 x a1
  703 x b1
  194 x a2
  187 x d
  127 x b2
   85 x f
   73 x a3
   40 x e
   29 x c2
   22 x c1
   22 x c3
   17 x v
   14 x c5
   13 x b3
   12 x a0
   12 x d1
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


### Modifier

The modifier is what occurs after the grapheme and after the `@` symbol, which should be digits and/or
lowercase letters except the `x`.

Sometimes modifiers occur inside a repeat, then we have stored the modifier in the feature
*modifierInner*, as in

```
7(N34@f)
```

Here is the frequency list of *modifier* and *modifierInner* values.

In [134]:
for (value, frequency) in F.modifier.freqList():
    print(f'{frequency:>5} x {value}')

  648 x g
  251 x t
   39 x n
    6 x r
    4 x s
    1 x c
    1 x v


In [135]:
for (value, frequency) in F.modifierInner.freqList():
    print(f'{frequency:>5} x {value}')

   25 x f
   15 x t
    1 x r
    1 x v


### Full grapheme overview

We make a frequency list of all full graphemes, i.e. the grapheme including variant, modifier, and prime.
We show them as they appear in transcriptions.

First we show on what node types primes, variants and modifiers occur.
We only deal with cases where they occur on signs, ignoring the cases where they occur on (sub)quads.

In [136]:
for feature in ('prime', 'variant', 'modifier'):
    nodeTypes = collections.Counter()
    for n in N():
        if Fs(feature).v(n):
            nodeTypes[F.otype.v(n)] += 1
    for (value, frequency) in nodeTypes.items():
        print(f'{feature:<10}: {frequency:>5} x {value}')

prime     :  4652 x case
prime     :   523 x column
prime     :     9 x sign
variant   : 32455 x sign
modifier  :   950 x sign


Now the write out the full grapheme distribution.

In [137]:
fullGraphemes = collections.Counter()

for n in F.otype.s('sign'):
    fullGrapheme = COMP.strFromSign(n)
    fullGraphemes[fullGrapheme] += 1
    
for (value, frequency) in sorted(fullGraphemes.items(), key=lambda x: (-x[1], x[0]))[0:20]:
    print(f'{frequency:>5} x {value}')
    
COMP.writeFreqs('grapheme-full', fullGraphemes.items(), 'full grapheme')

29618 x ...
17128 x 
12996 x 1(N01)
 6956 x X
 3081 x 2(N01)
 2606 x 1(N14)
 1849 x EN~a
 1603 x 3(N01)
 1357 x 2(N14)
 1308 x SZE~a
 1304 x 5(N01)
 1224 x GAL~a
 1119 x 4(N01)
 1069 x AN
 1045 x U4
 1001 x 1(N34)
  881 x SAL
  874 x GI
  854 x PAP~a
  801 x 1(N57)
There are 1529 full graphemes


Have a look at your `_temp` directory.

## End

Here ends the reburbished tutorial.
Below are fragments of the old tutorial.
These will be gone or reworked in the final version.

# Next steps

By now you have an impression how to compute around in the Hebrew Bible.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

## Search
Text-Fabric contains a flexible search engine, that does not only work for the BHSA data,
but also for data that you add to it.
There is a tutorial dedicated to [search](search.ipynb).
And if you already know MQL queries, you can build from that in
[searchFromMQL](searchFromMQL.ipynb).

## Explore additional data
The ETCBC has a few other repositories with data that work in conjunction with the BHSA data.
One of them you have already seen: 
[phono](https://github.com/ETCBC/phono),
for phonetic transcriptions.

There is also
[parallels](https://github.com/ETCBC/parallels)
for detecting parallel passages,
and
[valence](https://github.com/ETCBC/valence)
for studying patterns around verbs that determine their meanings.

## Add your own data
If you study the additional data, you can observe how that data is created and also
how it is turned into a text-fabric data module.
The last step is incredibly easy. You can write out every Python dictionary where the keys are numbers
and the values string or numbers as a Text-Fabric feature.
When you are creating data, you have already constructed those dictionaries, so writing
them out is just one method call.
See for example how the
[flowchart](https://github.com/ETCBC/valence/blob/master/programs/flowchart.ipynb#Add-sense-feature-to-valence-module)
notebook in valence writes out verb sense data.
![flow](images/valence.png)

You can then easily share your new features on GitHub, so that your colleagues everywhere 
can try it out for themselves.

## Export to Emdros MQL

[EMDROS](http://emdros.org), written by Ulrik Petersen,
is a text database system with the powerful *topographic* query language MQL.
The ideas are based on a model devised by Christ-Jan Doedens in
[Text Databases: One Database Model and Several Retrieval Languages](https://books.google.nl/books?id=9ggOBRz1dO4C).

Text-Fabric's model of slots, nodes and edges is a fairly straightforward translation of the models of Christ-Jan Doedens and Ulrik Petersen.

[SHEBANQ](https://shebanq.ancient-data.org) uses EMDROS to offer users to execute and save MQL queries against the Hebrew Text Database of the ETCBC.

So it is kind of logical and convenient to be able to work with a Text-Fabric resource through MQL.

If you have obtained an MQL dataset somehow, you can turn it into a text-fabric data set by `importMQL()`,
which we will not show here.

And if you want to export a Text-Fabric data set to MQL, that is also possible.

After the `Fabric(modules=...)` call, you can call `exportMQL()` in order to save all features of the
indicated modules into a big MQL dump, which can be imported by an EMDROS database.

# Clean caches

Text-Fabric pre-computes data for you, so that it can be loaded faster.
If the original data is updated, Text-Fabric detects it, and will recompute that data.

But there are cases, when the algorithms of Text-Fabric have changed, without any changes in the data, that you might
want to clear the cache of precomputed results.

There are two ways to do that:

* Locate the `.tf` directory of your dataset, and remove all `.tfx` files in it.
  This might be a bit awkward to do, because the `.tf` directory is hidden on Unix-like systems.
* Call `TF.clearCache()`, which does exactly the same.

It is not handy to execute the following cell all the time, that's why I have commented it out.
So if you really want to clear the cache, remove the comment sign below.

In [ ]:
# TF.clearCache()